In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [2]:
batch_size = 32
img_height = 16
img_width = 8

train_ds = tf.keras.utils.image_dataset_from_directory(
    "Dataset",
    validation_split=0.3,
    subset="training",
    seed=256,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "Dataset",
    validation_split=0.3,
    subset="validation",
    seed=256,
    image_size=(img_height, img_width),
    batch_size=batch_size)
    
normalization_layer = tf.keras.layers.Rescaling(1./255)

Found 6598 files belonging to 42 classes.
Using 4619 files for training.
Found 6598 files belonging to 42 classes.
Using 1979 files for validation.


In [3]:
class_names = val_ds.class_names
class_names

['0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '5',
 '51',
 '6',
 '7',
 '8',
 '9']

In [4]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

0.1764706 0.9843138


In [5]:
num_classes = len(os.listdir("Dataset"))

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(75, activation='relu'),
  # tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax)
])

In [6]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=5*1e-6)
# model.compile(optimizer=opt, 
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

from tensorflow.keras.callbacks import TensorBoard
import random
NAME = "basic"+str(random.randint(1,1000))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
# cd "C:\src\Python\Smart Clicker\V9\"
# tensorboard --logdir="logs/"

C:\Users\Admin\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50, callbacks=[tensorboard]
)


Epoch 1/50
145/145 [==============================] - 9s 53ms/step - loss: 2.7126 - accuracy: 0.3380 - val_loss: 1.4828 - val_accuracy: 0.6180
Epoch 2/50
145/145 [==============================] - 0s 3ms/step - loss: 1.0171 - accuracy: 0.7473 - val_loss: 0.7416 - val_accuracy: 0.8130
Epoch 3/50
145/145 [==============================] - 1s 4ms/step - loss: 0.5956 - accuracy: 0.8272 - val_loss: 0.4915 - val_accuracy: 0.8701
Epoch 4/50
145/145 [==============================] - 1s 4ms/step - loss: 0.4261 - accuracy: 0.8640 - val_loss: 0.3664 - val_accuracy: 0.9131
Epoch 5/50
145/145 [==============================] - 1s 5ms/step - loss: 0.3295 - accuracy: 0.8976 - val_loss: 0.2859 - val_accuracy: 0.9525
Epoch 6/50
145/145 [==============================] - 1s 5ms/step - loss: 0.2618 - accuracy: 0.9249 - val_loss: 0.2265 - val_accuracy: 0.9550
Epoch 7/50
145/145 [==============================] - 0s 3ms/step - loss: 0.2134 - accuracy: 0.9389 - val_loss: 0.1873 - val_accuracy: 0.9581
Epoch

In [8]:
model_name = "item_reader_seq.model"
model.save(model_name)

INFO:tensorflow:Assets written to: item_reader_seq.model\assets


In [9]:
import numpy as np
import cv2
X=[]
y=[]

for index, batch in enumerate(val_ds):
    X, y = batch
    X = X.numpy()
    y = y.numpy()
    
    y_predict = model.predict(X)
    y_predict = np.argmax(y_predict, axis=1)
    # for i in range(32):
    #     if y[i]!=y_predict[i]:
    #         print(index, i)
    #         cv2.imwrite(f"{random.randint(1, 100)}.png", X[i])
    
    cv2.imshow('image', X[0]) 
    cv2.imwrite("image.png", cv2.cvtColor(X[0], cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)
    
    print(class_names[y[0]], class_names[y_predict[0]])
    # print(y_predict)
    # break



51 51
26 26
27 27
7 7
32 32
19 19
3 3
34 34
19 19
11 11
37 37
1 1
8 8
39 39
38 38
7 7
3 3
33 33
17 17
28 28
7 7
28 28
37 37
26 26
8 8


KeyboardInterrupt: 